In [1]:
import glob
import csv
from pathlib import Path

from nltk.stem import PorterStemmer
ps = PorterStemmer()


Path("./Output/").mkdir(parents=True, exist_ok=True)
Path("./Output/Evaluation/").mkdir(parents=True, exist_ok=True)

file_writer = csv.writer(open('./Output/Inspec-PorterStemming.csv', 'w', encoding='UTF8', newline=''), delimiter='\t')
file_writer.writerow(['Top@', 'precision', 'recall', 'F1'])

In [4]:
# Get id of the test documents from the benchmarking path (../ake-datasets/../test)
# Modify the path (i.e., redirect to the ground-truth folder) to geth the gold standard keywords
test_docs= glob.glob('../ake-datasets/datasets/Inspec/test/*.xml')
test_docs= ['../Inspec/keys/'+doc.split('/')[-1][:-3]+'key' for doc in test_docs]

In [5]:
### Extact Matching ###

def extactMatcing(predictKeyphrases, groundKeyphrases):
    
    # compute porter stemming for predicted and groundtruth keyphrases    
    predKeyphrases_list= []
    for key in predictKeyphrases:
        predKeyphrases_list.append([ps.stem(k) for k in key.split()])
        
    groundKeyphrases_list= []
    for key in groundKeyphrases:
        groundKeyphrases_list.append([ps.stem(k) for k in key.split()])    
    
    exactMatch=True
    correctCount=0
                                   
    for predKey in predKeyphrases_list: 
                                   
        for goldKey in groundKeyphrases_list: 
            if len(predKey) != len(goldKey):
                continue
                                   
            for pred_w, gold_w in zip(predKey,goldKey): 
                    if pred_w !=gold_w:
                        exactMatch=False
                        break
        if exactMatch:
            correctCount+=1                        
                
    precision= correctCount/len(predictKeyphrases)
    recall = correctCount /len(groundKeyphrases)
    
    return precision, recall

In [ ]:
predicted_path='./Output/AKE/' # change this path to compute F1 evaluation based on exact-matching

#--- compute average recall, precision, F1 based on instance --#
num_of_sample = len(test_docs) # numbe of documents
sum_of_recall=0
sum_of_precision=0

#--- K= { 5, 10} ---#
k= 10

sum_of_recall=0
sum_of_precision=0

file_counts=0

for file in test_docs:
    
    # reading the ground-truth keyphrases as a list
    with open(file) as fileIn:
        groundtruth_keyphrases = fileIn.readlines()[:k]
        groundtruth_keyphrases= [keyphrase.replace('\n', '').replace('\t','') for keyphrase in groundtruth_keyphrases]
    fileIn.close()

    # reading the predicted keyphrases as a list
    fileName= file.split('/')[-1][:-3]+'txt'
    predictKey_file = Path(predicted_path+fileName)
    
    if not predictKey_file.is_file():
        continue
    
    file_counts+=1
    with open(predictKey_file) as fileIn:
        predicted_keyphrases = fileIn.readlines()[:k]
        predicted_keyphrases= [keyphrase.replace('\n', '') for keyphrase in predicted_keyphrases]
            
    #compute precision and recall-based on exact matching
    precision, recall= extactMatcing(predicted_keyphrases, groundtruth_keyphrases)
    
    sum_of_recall+= recall
    sum_of_precision+= precision
    
avg_recall= sum_of_recall/num_of_sample
avg_precision= sum_of_precision/num_of_sample

F1= 2*(avg_recall*avg_precision)/(avg_precision+avg_recall)

print (['Top@'+str(k), round(avg_precision, 3), round(avg_recall,3), round(F1, 3)])
#save evaluation results into csv file    
file_writer.writerow(['Top@'+str(k), round(avg_precision, 3), round(avg_recall,3), round(F1, 3)]) 
file_writer.flush()           